In [1]:

!pip install -q sklearn pandas numpy spacy nltk sentence-transformers scikit-learn joblib gradio
!python -m spacy download en_core_web_sm

import nltk
nltk.download('punkt')
nltk.download('stopwords')


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 88.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
import re
import os
import json
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
import joblib
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nlp = spacy.load("en_core_web_sm")

STOPWORDS = set(stopwords.words('english'))


In [3]:
# Small synthetic dataset of resume texts with labels (for demo & training)
data = [
    {"text":"Alice — data scientist with 3 years experience in Python, sklearn, pandas, SQL. Masters in CS. Worked at DataCorp. email: alice@mail.com", "label":"Data Scientist"},
    {"text":"Bob — frontend developer experienced in React, JavaScript, HTML, CSS. 2 years at WebInc. bob@mail.com", "label":"Frontend Developer"},
    {"text":"Carol — machine learning engineer. Hands-on with PyTorch, TensorFlow, model deployment, Docker. 4 years at MLWorks. carol@mail.com", "label":"ML Engineer"},
    {"text":"Dave — backend engineer skilled in Node.js, Express, MongoDB and AWS. 3 years. dave@mail.com", "label":"Backend Developer"},
    {"text":"Eve — full stack dev with React, Node, Python, and SQL. 5 years. eve@mail.com", "label":"Full Stack Developer"},
    # Add more samples to improve performance
]

df = pd.DataFrame(data)
df.head()


,text,label
0,Alice — data scientist with 3 years experience...,Data Scientist
1,"Bob — frontend developer experienced in React,...",Frontend Developer
2,Carol — machine learning engineer. Hands-on wi...,ML Engineer
3,"Dave — backend engineer skilled in Node.js, Ex...",Backend Developer
4,"Eve — full stack dev with React, Node, Python,...",Full Stack Developer


In [4]:
# Preprocessing and extraction utilities

email_re = re.compile(r'[\w\.-]+@[\w\.-]+\.\w+')
phone_re = re.compile(r'(\+?\d{1,3}[\s-])?(?:\(?\d{3}\)?[\s-]?)?\d{3}[\s-]?\d{4}')

def clean_text(text):
    text = text.replace('\n',' ').strip()
    # optional: lower, but keep case for NER sometimes
    return re.sub(r'\s+',' ', text)

def extract_email(text):
    m = email_re.search(text)
    return m.group(0) if m else None

def extract_phone(text):
    m = phone_re.search(text)
    return m.group(0) if m else None

def extract_entities(text):
    doc = nlp(text)
    orgs = [ent.text for ent in doc.ents if ent.label_ in ('ORG',)]
    edu = [ent.text for ent in doc.ents if ent.label_ in ('ORG','NORP','GPE','PERSON') and 'University' in ent.text or 'College' in ent.text]  # naive
    # extract PERSON as potential name
    persons = [ent.text for ent in doc.ents if ent.label_ == 'PERSON']
    return {"orgs": orgs, "education": edu, "persons": persons}

def extract_skills(text, skillset=None):
    # naive skill matching: provide a list of skills to check presence
    if skillset is None:
        skillset = ["python","pandas","sql","scikit-learn","tensorflow","pytorch","react","node","docker","aws","html","css","javascript"]
    text_low = text.lower()
    found = [s for s in skillset if s.lower() in text_low]
    return found

# quick test
sample = df.loc[0, "text"]
print(clean_text(sample))
print("email:", extract_email(sample))
print("phone:", extract_phone(sample))
print("entities:", extract_entities(sample))
print("skills:", extract_skills(sample))


Alice — data scientist with 3 years experience in Python, sklearn, pandas, SQL. Masters in CS. Worked at DataCorp. email: alice@mail.com
email: alice@mail.com
phone: None
entities: {'orgs': ['Alice', 'pandas', 'SQL', 'CS'], 'education': [], 'persons': []}
skills: ['python', 'pandas', 'sql']


In [6]:
# Build TF-IDF vectorizer and add simple skill-count as a numeric feature.
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion

class SkillCount(BaseEstimator, TransformerMixin):
    def __init__(self, skillset=None):
        self.skillset = skillset or ["python","pandas","sql","scikit-learn","tensorflow","pytorch","react","node","docker","aws","html","css","javascript"]
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        counts = []
        for text in X:
            found = sum(1 for s in self.skillset if s.lower() in text.lower())
            counts.append([found])
        return np.array(counts)

# Vectorizer and pipeline
tfidf = TfidfVectorizer(ngram_range=(1,2), max_features=3000)

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

# We'll create a combined pipeline manually:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

# Prepare X and y
X = df['text'].apply(clean_text).values
y = df['label'].values

# Simple approach: combine tfidf features and skill-count using FeatureUnion
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import FunctionTransformer

tfidf_transformer = Pipeline([
    ('tfidf', tfidf)
])

skill_transformer = Pipeline([
    ('skillcount', SkillCount())
])

feature_union = FeatureUnion([
    ('tfidf', tfidf_transformer),
    ('skills', skill_transformer)
])

pipeline = Pipeline([
    ('features', feature_union),
    ('clf', LogisticRegression(max_iter=1000))
])

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
pipeline.fit(X_train, y_train)

# Evaluate
preds = pipeline.predict(X_test)
print("Accuracy:", accuracy_score(y_test, preds))
print(classification_report(y_test, preds))

Accuracy: 0.0
                      precision    recall  f1-score   support

      Data Scientist       0.00      0.00      0.00       0.0
  Frontend Developer       0.00      0.00      0.00       1.0
Full Stack Developer       0.00      0.00      0.00       1.0
         ML Engineer       0.00      0.00      0.00       0.0

            accuracy                           0.00       2.0
           macro avg       0.00      0.00      0.00       2.0
        weighted avg       0.00      0.00      0.00       2.0



/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py

In [7]:

fitted_tfidf = pipeline.named_steps['features'].transformer_list[0][1].named_steps['tfidf']

def ats_score(resume_text, job_keywords):
    # job_keywords: list of important keywords from JD
    found = sum(1 for k in job_keywords if k.lower() in resume_text.lower())
    score = found / max(1, len(job_keywords))
    return float(score)

def resume_jd_similarity(resume, jd_text):
    # use TF-IDF vectors and cosine similarity
    vectors = fitted_tfidf.transform([clean_text(resume), clean_text(jd_text)])
    sim = cosine_similarity(vectors[0], vectors[1])[0][0]
    return float(sim)

# Demo
jd = "Looking for Data Scientist with Python, pandas, SQL, machine learning and model deployment experience."
resume = df.loc[0, "text"]
print("ATS score:", ats_score(resume, ["python","sql","machine learning","pandas","docker"]))
print("Similarity:", resume_jd_similarity(resume, jd))


ATS score: 0.6
Similarity: 0.37258203768963566


In [8]:
def analyze_resume(resume_text, job_description=None, job_keywords=None):
    t = clean_text(resume_text)
    email = extract_email(t)
    phone = extract_phone(t)
    entities = extract_entities(t)
    skills = extract_skills(t)
    # classification
    pred_role = pipeline.predict([t])[0]
    # ATS score if keywords provided
    ats = None
    if job_keywords:
        ats = ats_score(t, job_keywords)
    # similarity if JD provided
    sim = None
    if job_description:
        sim = resume_jd_similarity(t, job_description)
    # suggestions: missing top keywords from JD
    suggestions = []
    if job_keywords:
        missing = [k for k in job_keywords if k.lower() not in t.lower()]
        suggestions = missing[:10]
    return {
        "email": email,
        "phone": phone,
        "entities": entities,
        "skills": skills,
        "predicted_role": pred_role,
        "ats_score": ats,
        "similarity": sim,
        "suggestions": suggestions
    }

# Try
print(analyze_resume(df.loc[0,"text"], job_description=jd, job_keywords=["python","sql","machine learning","pandas","docker"]))


{'email': 'alice@mail.com', 'phone': None, 'entities': {'orgs': ['Alice', 'pandas', 'SQL', 'CS'], 'education': [], 'persons': []}, 'skills': ['python', 'pandas', 'sql'], 'predicted_role': 'Data Scientist', 'ats_score': 0.6, 'similarity': 0.37258203768963566, 'suggestions': ['machine learning', 'docker']}


/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [9]:
# Save the whole pipeline (includes features and classifier)
model_path = "resume_pipeline.joblib"
joblib.dump(pipeline, model_path)
print("Saved model to", model_path)


Saved model to resume_pipeline.joblib


In [10]:
from sentence_transformers import SentenceTransformer
# choose a small model for Colab memory; this may download ~100-200MB
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

def embed_similarity(resume, jd):
    v1 = embed_model.encode([clean_text(resume)])
    v2 = embed_model.encode([clean_text(jd)])
    return float(cosine_similarity([v1[0]],[v2[0]])[0][0])

# demo
print("Embedding similarity:", embed_similarity(resume, jd))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding similarity: 0.6887547969818115


In [12]:
import gradio as gr

def gradio_analyze(resume_text, job_desc, keywords_text):
    keywords = [k.strip() for k in keywords_text.split(",")] if keywords_text else None
    out = analyze_resume(resume_text, job_description=job_desc, job_keywords=keywords)
    # Build readable output
    summary = {
        "Predicted Role": out["predicted_role"],
        "Email": out["email"],
        "Phone": out["phone"],
        "Skills Found": ", ".join(out["skills"]),
        "ATS Score": out["ats_score"],
        "Similarity": out["similarity"],
        "Suggestions (missing keywords)": ", ".join(out["suggestions"])
    }
    return json.dumps(summary, indent=2), out["entities"]

iface = gr.Interface(
    fn=gradio_analyze,
    inputs=[
        gr.Textbox(lines=10, label="Paste Resume Text"),
        gr.Textbox(lines=5, label="Paste Job Description (optional)"),
        gr.Textbox(lines=1, label="Comma-separated Job Keywords (optional)")
    ],
    outputs=[
        gr.Textbox(label="Analysis Summary (JSON)"),
        gr.JSON(label="Named Entities")
    ],
    examples=[
        [df.loc[0,"text"], jd, "python,sql,pandas,machine learning"]
    ],
    title="Resume Analyzer (Local Demo)"
)

# Launch the demo in colab — will give a public link
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fdbf2eae0b8aabb099.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
